# AI Workshop - Lab 3-1a: Optimization
## Supply Chain Logistics Optimization using Google OR-Tools

This notebook will guide you through using Google OR-Tools to solve a supply chain optimization problem. We'll use Linear Programming (LP) to minimize transportation costs in a supply chain.

---

## Problem Overview

Imagine a Canadian company that needs to transport goods from its distribution centers to retail stores across the country. Each distribution center has a limited supply, and each store requires a certain amount of goods. Additionally, transporting goods incurs costs, which depend on the route between the distribution center and the store. Our goal is to determine the optimal transportation plan to minimize total costs.

We'll use Google OR-Tools to solve this problem. OR-Tools is an open-source library for combinatorial optimization, which includes linear programming, constraint programming, and vehicle routing.

In [ ]:
!pip install -q ortools

In [ ]:
from ortools.linear_solver import pywraplp

## Problem Data

We'll use the following data for our problem:

- **Distribution Centers**: The supply locations.
- **Stores**: The demand locations.
- **Costs**: The transportation costs between distribution centers and stores.
- **Supplies**: The available supply at each distribution center.
- **Demands**: The required demand at each store.

We'll represent this data using Python dictionaries.

In [ ]:
# Data

# Distribution Centers - a list of supply locations
distribution_centers = ['Vancouver Distribution Center', 'Toronto Distribution Center']

# Stores - retail locations across Canada
stores = ['Calgary Superstore', 'Montreal Superstore', 'Ottawa Superstore']

# Transportation Costs (in dollars per unit)
costs = {
    ('Vancouver Distribution Center', 'Calgary Superstore'): 2,
    ('Vancouver Distribution Center', 'Montreal Superstore'): 5,
    ('Vancouver Distribution Center', 'Ottawa Superstore'): 4,
    ('Toronto Distribution Center', 'Calgary Superstore'): 4,
    ('Toronto Distribution Center', 'Montreal Superstore'): 2,
    ('Toronto Distribution Center', 'Ottawa Superstore'): 1,
}

# Supplies at each distribution center (in units)
supplies = {
    'Vancouver Distribution Center': 120,
    'Toronto Distribution Center': 150,
}

# Demands at each store (in units)
demands = {
    'Calgary Superstore': 80,
    'Montreal Superstore': 100,
    'Ottawa Superstore': 70,
}

We have two distribution centers in Vancouver and Toronto, and three stores in Calgary, Montreal, and Ottawa. The transportation costs between each distribution center-store pair are given in dollars per unit. The available supplies at the distribution centers and the required demands at the stores are also provided. We can visualize the problem using the code below:

In [ ]:
# Import necessary libraries
import matplotlib.pyplot as plt
import networkx as nx

# Positions for the nodes (approximate locations on the map)
positions = {
    'Vancouver Distribution Center': (-123.1, 49.3),  # Vancouver coordinates
    'Toronto Distribution Center': (-79.4, 43.7),     # Toronto coordinates
    'Calgary Superstore': (-114.1, 51.0),             # Calgary coordinates
    'Montreal Superstore': (-73.6, 45.5),             # Montreal coordinates
    'Ottawa Superstore': (-75.7, 45.4),               # Ottawa coordinates
}

# Create the graph
G = nx.DiGraph()

# Add nodes for distribution centers and stores
G.add_nodes_from(distribution_centers)
G.add_nodes_from(stores)

# Add edges with costs as weights
for (dc, store), cost in costs.items():
    G.add_edge(dc, store, weight=cost)

# Extract positions for plotting
node_positions = {node: positions[node] for node in G.nodes()}

# Create node labels including supplies and demands
node_labels = {}
for node in G.nodes():
    if node in distribution_centers:
        supply = supplies[node]
        node_labels[node] = f"{node}\n(Supply: {supply})"
    elif node in stores:
        demand = demands[node]
        node_labels[node] = f"{node}\n(Demand: {demand})"

# Assign colors and shapes
dc_nodes = [node for node in G.nodes() if node in distribution_centers]
store_nodes = [node for node in G.nodes() if node in stores]

# Draw the graph
plt.figure(figsize=(12, 8))

# Draw distribution centers
nx.draw_networkx_nodes(
    G,
    node_positions,
    nodelist=dc_nodes,
    node_color='skyblue',
    node_shape='s',
    node_size=600,
    label='Distribution Centers'
)

# Draw stores
nx.draw_networkx_nodes(
    G,
    node_positions,
    nodelist=store_nodes,
    node_color='lightgreen',
    node_shape='o',
    node_size=400,
    label='Stores'
)

# Draw edges with labels
edge_labels = nx.get_edge_attributes(G, 'weight')
nx.draw_networkx_edges(
    G,
    node_positions,
    arrowstyle='->',
    arrowsize=15,
    edge_color='gray'
)

# Draw node labels with supplies and demands
nx.draw_networkx_labels(
    G,
    node_positions,
    labels=node_labels,
    font_size=9
)

# Draw edge labels (costs)
edge_labels_formatted = {(u, v): f"${d['weight']}" for u, v, d in G.edges(data=True)}
nx.draw_networkx_edge_labels(
    G,
    node_positions,
    edge_labels=edge_labels_formatted,
    font_color='red',
    font_size=8,
    label_pos=0.3
)

# Add legend
plt.legend(scatterpoints=1, fontsize=12)

# Set plot title and axes
plt.title('Supply Chain Network Visualization with Supplies and Demands')
plt.axis('off')

# Show the plot
plt.show()


## Model

Next, we'll define the optimization model using Google OR-Tools. We'll create a linear programming model to minimize the total transportation costs.

Google OR-Tools provides a Python API for defining linear programming models. We can create variables, constraints, and the objective function using this API. We'll be using the GLOP linear solver, which is a high-performance solver for linear programming problems.

Let's define the model.

In [ ]:
# Model

# Create the linear solver with the GLOP backend
solver = pywraplp.Solver.CreateSolver('GLOP')

## Defining the Decision Variables

In this step, we define the "decision variables" for our optimization model. These variables answer the question:

**"How many goods should be transported from each distribution center to each store?"**

For every possible route between a distribution center and a store, we create a variable to represent the number of goods transported along that route.

### Key Points:
- Each variable corresponds to a specific route (e.g., Vancouver Distribution Center to Calgary Superstore).
- The variables can only take non-negative values:
  - A value of `0` means no goods are transported along that route.
  - A positive value indicates the quantity of goods transported.
- These variables will later help us calculate the total transportation costs and ensure supply and demand constraints are met.

Here’s the code to define these variables:

In [ ]:
# Create a variable for each distribution center-store pair
variables = {}
for dc in distribution_centers:
    for store in stores:
        variables[dc, store] = solver.NumVar(
            0,                 # Minimum value to be sent from one centre to one store
            supplies[dc],      # Maximum value to be sent from one centre to one store - a dc cannot send more than its supply
            f'{dc} to {store}' # Name of the variable
        )

Notice that we defined the **bounds** of each variable to be between 0 and the total number of supplies available at the distribution center. This ensures that the total amount of goods transported from each distribution center does not exceed the available supply.

---

## Defining the Constraints

In this step, we add the **constraints** to our optimization model. Constraints are rules that limit the possible solutions, ensuring that:

1. **Supply Constraints:**
   The total amount of goods transported from each distribution center cannot exceed the supply available at that center.
   This ensures that we don't ship more goods from a distribution center than it has in stock. Note that above we set a restriction that one dc cannot send one store more than its supply. Here we are saying the **total** amount of goods sent from a dc cannot exceed its supply.

2. **Demand Constraints:**
   The total amount of goods received at each store must meet or exceed the demand of that store.
   This ensures that every store gets enough goods to satisfy its requirements.

---

### **Supply Constraints**

For each distribution center:
- We sum the goods transported from that center to all stores.
- We ensure this total does not exceed the center's available supply.

### **Demand Constraints**

For each store:
- We sum the goods received at that store from all distribution centers.
- We ensure this total meets or exceeds the store's demand.

In [ ]:
# Supply Constraints: total goods transported from each distribution center ≤ available supply
for dc in distribution_centers:
    total_supplies_sent = solver.Sum(variables[dc, store] for store in stores)
    solver.Add(total_supplies_sent <= supplies[dc])

# Demand Constraints: total goods received at each store ≥ required demand
for store in stores:
    total_demands_met = solver.Sum(variables[dc, store] for dc in distribution_centers)
    solver.Add(total_demands_met >= demands[store])

## Defining the Objective Function

The **objective function** is the most important part of the optimization model. It tells the solver what we are trying to achieve.

In this case, our goal is to **minimize the total transportation costs**. The transportation cost for each route is calculated by multiplying:

- The **amount of goods transported** along the route.
- The **cost per unit** for that route.

The total transportation cost is the sum of costs across all routes between distribution centers and stores.

In [ ]:
# Objective Function: minimize total transportation costs
for dc in distribution_centers:
    for store in stores:
        solver.Minimize(variables[dc, store] * costs[dc, store]) # The cost for each route is the amount of goods transported multiplied by the cost per unit

Now that we've defined the model, we can run the solver to find the optimal solution! We'll call the `Solve` method on the solver object to solve the linear programming problem.

In [ ]:
# Solve the model
status = solver.Solve()

In [ ]:
# Check if the problem has an optimal solution
if status == pywraplp.Solver.OPTIMAL:
    print('Optimal Solution Found')
else:
    print('The problem does not have an optimal solution')

Fantastic - the solver found an optimal solution! Now let's print out the optimal transportation plan and the total transportation costs.

In [ ]:
total_cost = 0
print('Optimal Transportation Plan:')
for dc in distribution_centers:
    for store in stores:
        amount = variables[dc, store].solution_value()
        if amount > 0:
            cost = amount * costs[dc, store]
            total_cost += cost
            print(f'Transport {amount:.0f} units from {dc} to {store} at a cost of ${cost:.0f}')
print(f'\nTotal Transportation Cost: ${total_cost:.0f}')

Surprisingly, this solution has us send none of the units from the Vancouver centre to Calgary, even though those cities are much closer! Remember that the values we have set here are invented, and probably don't accurately represent the real costs of shipping between these cities.

# Summary

In this notebook, we used Google OR-Tools to solve a supply chain optimization problem. We defined a linear programming model to minimize transportation costs in a supply chain, considering the available supply at each warehouse and the required demand at each store. We then used the GLOP linear solver to find the optimal transportation plan that minimizes total costs.

You can modify the data and constraints to test different scenarios and see how the optimal solution changes. Linear programming is a powerful tool for optimization problems, and Google OR-Tools makes it easy to define and solve these models in Python.

